<a href="https://colab.research.google.com/github/sjdoescoding/CE889/blob/main/Final_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
# import scipy.stats as stats
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# data = None
# test_data = None
# import os
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         path = os.path.join(dirname, filename)
#         if filename == 'train.csv':
#             data = pd.read_csv(path)
#         if filename == 'test.csv':
#             test_data = pd.read_csv(path)
#         if filename == 'store.csv':
#             store_data = pd.read_csv(path)
        

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import scipy.stats as stats
import matplotlib.pyplot as plt

In [52]:
from google.colab import drive #mounting to drive
drive.mount('/content/drive', force_remount=True)

# Reading Zip
from zipfile import ZipFile
file_name = '/content/drive/My Drive/CE889/Group_Assignment/rossmann-store-sales.zip' 
with ZipFile(file_name, 'r') as zip:
    zip.printdir()
    zip.extractall()
    print('Done')

store_data = pd.read_csv('store.csv')

#Assigning data set to df
data = pd.read_csv('train.csv')

#Assigning Test data to df
test_data = pd.read_csv('test.csv')

Mounted at /content/drive
File Name                                             Modified             Size
sample_submission.csv                          2019-12-11 03:57:00       317611
store.csv                                      2019-12-11 03:57:00        45010
test.csv                                       2019-12-11 03:57:00      1427425
train.csv                                      2019-12-11 03:57:02     38057952
Done


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
# Join train set and store set, test set and store set, inner refers to using the common keys
# on refers to the label
new_train = data.merge(store_data, how='inner', on='Store')
new_test = test_data.merge(store_data, how='inner', on='Store')

In [54]:
new_train['Date'] = pd.to_datetime(new_train['Date'])
new_train['Year'] = new_train.Date.dt.year
new_train['Month'] = new_train.Date.dt.month
new_train['Day'] = new_train.Date.dt.day

new_test['Date'] = pd.to_datetime(new_test['Date'])
new_test['Year'] = new_train.Date.dt.year
new_test['Month'] = new_test.Date.dt.month
new_test['Day'] = new_test.Date.dt.day

In [55]:
new_train["CompetitionDistance"]=new_train["CompetitionDistance"].fillna(99999)
new_test["CompetitionDistance"]=new_test["CompetitionDistance"].fillna(99999)

In [56]:
 # convert assortment and storetype and stateholiday in training data to int
cleanup = {"Assortment": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup, inplace=True)
new_train['Assortment'] = new_train['Assortment'].astype(int)

cleanup2 = {"StoreType": {"a":0, "b":1, "c":2, "d":3}}
new_train.replace(cleanup2, inplace=True)
new_train['StoreType'] = new_train['StoreType'].astype(int)

cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup3, inplace=True)
new_train['StateHoliday'] = new_train['StateHoliday'].astype(int)

In [57]:
 # convert assortment and storetype in testing data to int
cleanup = {"Assortment": {"a":0, "b":1, "c":2}}
new_test.replace(cleanup, inplace=True)
new_test['Assortment'] = new_test['Assortment'].astype(int)

cleanup2 = {"StoreType": {"a":0, "b":1, "c":2, "d":3}}
new_test.replace(cleanup2, inplace=True)
new_test['StoreType'] = new_test['StoreType'].astype(int)

cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup3, inplace=True)
new_train['StateHoliday'] = new_train['StateHoliday'].astype(int)

In [58]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 21 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1017209 non-null  int64         
 1   DayOfWeek                  1017209 non-null  int64         
 2   Date                       1017209 non-null  datetime64[ns]
 3   Sales                      1017209 non-null  int64         
 4   Customers                  1017209 non-null  int64         
 5   Open                       1017209 non-null  int64         
 6   Promo                      1017209 non-null  int64         
 7   StateHoliday               1017209 non-null  int64         
 8   SchoolHoliday              1017209 non-null  int64         
 9   StoreType                  1017209 non-null  int64         
 10  Assortment                 1017209 non-null  int64         
 11  CompetitionDistance        1017209 no

In [59]:
 # Null value exists in open, fill with one, cuz stores would open except from holidays
new_train['Open'] = new_train['Open'].fillna(1)
new_train['Open'] = new_train['Open'].astype(int)


new_test['Open'] = new_test['Open'].fillna(1)
new_test['Open' ]= new_test['Open'].astype(int)

In [60]:
 # fill nan with month 0, years in the far future
new_train['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
new_train['CompetitionOpenSinceYear'].fillna(2100, inplace=True)

new_train['Promo2SinceWeek'].fillna(1, inplace=True)
new_train['Promo2SinceYear'].fillna(2100, inplace=True)
new_train['PromoInterval'].fillna(0, inplace=True)

new_test['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
new_test['CompetitionOpenSinceYear'].fillna(2100, inplace=True)

new_test['Promo2SinceWeek'].fillna(1, inplace=True)
new_test['Promo2SinceYear'].fillna(2100, inplace=True)
new_test['PromoInterval'].fillna(0, inplace=True)

In [61]:
 # combine competition month and year and promo week year to form days
new_train['CompetitionDays'] = 365 * (new_train['Year'] - new_train['CompetitionOpenSinceYear']) + new_train['Month'] - new_train['CompetitionOpenSinceMonth']

new_train['PromoOpenDays'] = 365 * (new_train['Year'] - new_train['Promo2SinceYear']) + ((new_train['Day'] - 7 * new_train['Promo2SinceWeek']))

new_test['CompetitionDays'] = 365 * (new_test['Year'] - new_test['CompetitionOpenSinceYear']) + new_test['Month'] - new_test['CompetitionOpenSinceMonth']

new_test['PromoOpenDays'] = 365 * (new_test['Year'] - new_test['Promo2SinceYear']) + ((new_test['Day'] - 7 * new_test['Promo2SinceWeek']))

In [62]:
new_train.drop(columns = ['CompetitionOpenSinceMonth'], inplace = True)
new_train.drop(columns =['CompetitionOpenSinceYear'], inplace = True)
new_train.drop(columns =['Promo2SinceWeek'], inplace = True)
new_train.drop(columns =['Promo2SinceYear'], inplace = True)
new_train.drop(columns =['Date'], inplace = True)

In [63]:
new_train = new_train.query('Open==1')
new_test = new_test.query('Open==1')

In [64]:
# dropping customers
new_train.drop(columns=['Customers'],inplace = True)


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [65]:
new_test

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,CompetitionDays,PromoOpenDays
0,1,1,4,2015-09-17,1,1,0,0,2,0,...,2008.0,0,1.0,2100.0,0,2015,9,17,2555.0,-31015.0
1,857,1,3,2015-09-16,1,1,0,0,2,0,...,2008.0,0,1.0,2100.0,0,2015,9,16,2555.0,-31016.0
2,1713,1,2,2015-09-15,1,1,0,0,2,0,...,2008.0,0,1.0,2100.0,0,2015,9,15,2555.0,-31017.0
3,2569,1,1,2015-09-14,1,1,0,0,2,0,...,2008.0,0,1.0,2100.0,0,2015,9,14,2555.0,-31018.0
5,4281,1,6,2015-09-12,1,0,0,0,2,0,...,2008.0,0,1.0,2100.0,0,2015,9,12,2555.0,-31020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41082,36808,1115,4,2015-08-06,1,1,0,1,3,2,...,2100.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,8,6,-31748.0,217.0
41083,37664,1115,3,2015-08-05,1,1,0,1,3,2,...,2100.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,8,5,-31748.0,216.0
41084,38520,1115,2,2015-08-04,1,1,0,1,3,2,...,2100.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,8,4,-31748.0,215.0
41085,39376,1115,1,2015-08-03,1,1,0,1,3,2,...,2100.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,8,3,-31748.0,214.0


In [66]:
new_train.dtypes

Store                    int64
DayOfWeek                int64
Sales                    int64
Open                     int64
Promo                    int64
StateHoliday             int64
SchoolHoliday            int64
StoreType                int64
Assortment               int64
CompetitionDistance    float64
Promo2                   int64
PromoInterval           object
Year                     int64
Month                    int64
Day                      int64
CompetitionDays        float64
PromoOpenDays          float64
dtype: object

In [67]:
new_train = pd.get_dummies(new_train, columns = ['PromoInterval'])
new_test = pd.get_dummies(new_test, columns = ['PromoInterval'])

In [68]:
new_train.dtypes

Store                               int64
DayOfWeek                           int64
Sales                               int64
Open                                int64
Promo                               int64
StateHoliday                        int64
SchoolHoliday                       int64
StoreType                           int64
Assortment                          int64
CompetitionDistance               float64
Promo2                              int64
Year                                int64
Month                               int64
Day                                 int64
CompetitionDays                   float64
PromoOpenDays                     float64
PromoInterval_0                     uint8
PromoInterval_Feb,May,Aug,Nov       uint8
PromoInterval_Jan,Apr,Jul,Oct       uint8
PromoInterval_Mar,Jun,Sept,Dec      uint8
dtype: object

In [69]:
x = new_train.drop(["Sales"], axis = 1)
y = new_train["Sales"]

In [70]:
x = x.to_numpy()
y = y.to_numpy()

In [71]:
len(x[0])

19

In [72]:
cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_test.replace(cleanup3, inplace=True)
new_test['StateHoliday'] = new_test['StateHoliday'].astype(int)

In [73]:
change_days = new_train['CompetitionDays']
change_days=list(change_days)
for i in range(len(change_days)):
    if change_days[i]<0:
        change_days[i]=0
new_train['CompetitionDays']=change_days
changepromo = new_train['PromoOpenDays']
changepromo=list(changepromo)
for i in range(len(changepromo)):
    if changepromo[i]<0:
        changepromo[i]=0
new_train['PromoOpenDays']=changepromo
new_train

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,Month,Day,CompetitionDays,PromoOpenDays,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
0,1,5,5263,1,1,0,1,2,0,1270.0,0,2015,7,31,2553.0,0.0,1,0,0,0
1,1,4,5020,1,1,0,1,2,0,1270.0,0,2015,7,30,2553.0,0.0,1,0,0,0
2,1,3,4782,1,1,0,1,2,0,1270.0,0,2015,7,29,2553.0,0.0,1,0,0,0
3,1,2,5011,1,1,0,1,2,0,1270.0,0,2015,7,28,2553.0,0.0,1,0,0,0
4,1,1,6102,1,1,0,1,2,0,1270.0,0,2015,7,27,2553.0,0.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017202,1115,1,6905,1,1,0,1,3,2,5350.0,1,2013,1,7,0.0,218.0,0,0,0,1
1017204,1115,6,4771,1,0,0,1,3,2,5350.0,1,2013,1,5,0.0,216.0,0,0,0,1
1017205,1115,5,4540,1,0,0,1,3,2,5350.0,1,2013,1,4,0.0,215.0,0,0,0,1
1017206,1115,4,4297,1,0,0,1,3,2,5350.0,1,2013,1,3,0.0,214.0,0,0,0,1


In [74]:
change_days = new_test['CompetitionDays']
change_days=list(change_days)
for i in range(len(change_days)):
    if change_days[i]<0:
        change_days[i]=0
new_test['CompetitionDays']=change_days
changepromo = new_test['PromoOpenDays']
changepromo=list(changepromo)
for i in range(len(changepromo)):
    if changepromo[i]<0:
        changepromo[i]=0
new_test['PromoOpenDays']=changepromo
new_test

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2SinceYear,Year,Month,Day,CompetitionDays,PromoOpenDays,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
0,1,1,4,2015-09-17,1,1,0,0,2,0,...,2100.0,2015,9,17,2555.0,0.0,1,0,0,0
1,857,1,3,2015-09-16,1,1,0,0,2,0,...,2100.0,2015,9,16,2555.0,0.0,1,0,0,0
2,1713,1,2,2015-09-15,1,1,0,0,2,0,...,2100.0,2015,9,15,2555.0,0.0,1,0,0,0
3,2569,1,1,2015-09-14,1,1,0,0,2,0,...,2100.0,2015,9,14,2555.0,0.0,1,0,0,0
5,4281,1,6,2015-09-12,1,0,0,0,2,0,...,2100.0,2015,9,12,2555.0,0.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41082,36808,1115,4,2015-08-06,1,1,0,1,3,2,...,2012.0,2013,8,6,0.0,217.0,0,0,0,1
41083,37664,1115,3,2015-08-05,1,1,0,1,3,2,...,2012.0,2013,8,5,0.0,216.0,0,0,0,1
41084,38520,1115,2,2015-08-04,1,1,0,1,3,2,...,2012.0,2013,8,4,0.0,215.0,0,0,0,1
41085,39376,1115,1,2015-08-03,1,1,0,1,3,2,...,2012.0,2013,8,3,0.0,214.0,0,0,0,1


In [75]:
#@title Your Title Here
# class Autoencoder(torch.nn.Module):
    
#     def __init__(self, n_inp, n_hidden, name):
#         super().__init__()
        
#         print(n_inp, "->", n_hidden)
        
#         self.hidden = nn.Linear(in_features=n_inp, out_features=n_hidden)
#         print(self.hidden)
#         self.output = nn.Linear(in_features=n_hidden, out_features=n_inp)
#         print(self.output)
#         print(n_inp, "->", n_hidden)
#         self.is_learning = True
    
#     def set_learning(self, value):
#         self.is_learning = value
        
#     def forward(self, x, function=torch.relu):
        
#         output = function(self.hidden(x))
        
#         if self.is_learning:
#             output = function(self.output(output))
        
#         return output

In [76]:
#@title Your Title Here
# def trainAutoencoder(epochs: int, size: int, _input = x):
    
#         layer = Autoencoder(len(_input[0]), size, epochs)

#         optimizer = torch.optim.Adam(layer.parameters(),lr=0.0001)

#         # mean-squared error loss
#         criterion = nn.MSELoss()

#         for epoch in range(epochs):
#             loss = 0

#             train_loader = torch.utils.data.DataLoader(_input, batch_size=10000, shuffle=True, num_workers=4, pin_memory=True)

#             for batch_features in train_loader:
                
#                 batch_features = batch_features.view(-1, len(_input[0]))
#                 optimizer.zero_grad()
#                 outputs = layer(batch_features.float())
#                 train_loss = criterion(outputs, batch_features.float())
#                 train_loss.backward()
#                 optimizer.step()
#                 loss += train_loss.item()

#             loss = loss / len(train_loader)
#             print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))
#         layer.set_learning(False)
#         return layer

In [77]:
#@title Your Title Here
# def feedforward(models: [], x):
    
#     output = torch.tensor(x).float()

#     for model in models:
#         output1 = model(output)
#         output = output1
        
#     return output

In [78]:
#@title Your Title Here
# def trainLayers(epochs: [], size_of_layer: [], _input):
#     assert len(epochs) == len(size_of_layer)
#     models = []
#     print(size_of_layer[0])
#     hidden_layer = trainAutoencoder(epochs[0], size_of_layer[0], _input)
#     models.append(hidden_layer)
    
#     for index, size in enumerate(size_of_layer):
#         if index == 0:
#             continue
#         h_layer_2 = trainAutoencoder(epochs[index], size_of_layer[index], feedforward(models, _input).detach())
#         models.append(h_layer_2)
    
#     return models

# models = trainLayers([2, 2, 2], [10, 5, 1], x)

# Fitting the data to the model

In [79]:
# Normalization and scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Scaling the data 
scaler = MinMaxScaler()
scaler.fit_transform(x,y)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [80]:
# Shape of sets
X_train.shape, X_test.shape, y_test.shape, y_train.shape

((675513, 19), (168879, 19), (168879,), (675513,))

In [81]:
# Resizing Shape of test and train set to 3d arrays - you need 3d array for RNN to work, needs to be changed depending on size of train test split and if data changes
# The size of shapes should reflect the split data 

X_train = X_train.reshape(len(X_train), 1, len(x[0])) # Reshape of the data must be the same size as the split data
X_test = X_test.reshape(len(X_test), 1, len(x[0]))

In [82]:
# Look above for reference of size
X_train.shape, X_test.shape, y_test.shape, y_train.shape

((675513, 1, 19), (168879, 1, 19), (168879,), (675513,))

# RNN/LSTM Model

In [83]:
# Importing Tensorflow library
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam


## LSTM/RNN Model
model1 = Sequential()
model1.add(InputLayer((1, len(x[0])))) #Note the inputs, (x1,x2) x1 refers to the batch size which is equal to the 3d shape above.
model1.add(LSTM(32, activation='relu',recurrent_activation='tanh',))
model1.add(Dropout(0.3))
model1.add(Dense(16, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 32)                6656      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [84]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=1e-4), metrics=[RootMeanSquaredError()])

In [85]:
model1.fit(X_train, y_train, epochs=20, callbacks=[cp1])

Epoch 1/20
21095/21110 [============================>.] - ETA: 0s - loss: 13834337.0000 - root_mean_squared_error: 3719.4539

21110/21110 [==============================] - 57s 3ms/step - loss: 13832376.0000 - root_mean_squared_error: 3719.1902
Epoch 2/20
21110/21110 [==============================] - ETA: 0s - loss: 10664126.0000 - root_mean_squared_error: 3265.5974

21110/21110 [==============================] - 60s 3ms/step - loss: 10664126.0000 - root_mean_squared_error: 3265.5974
Epoch 3/20
21093/21110 [============================>.] - ETA: 0s - loss: 10265491.0000 - root_mean_squared_error: 3203.9805

21110/21110 [==============================] - 55s 3ms/step - loss: 10266141.0000 - root_mean_squared_error: 3204.0820
Epoch 4/20
21110/21110 [==============================] - ETA: 0s - loss: 10110209.0000 - root_mean_squared_error: 3179.6555

21110/21110 [==============================] - 54s 3ms/step - loss: 10110209.0000 - root_mean_squared_error: 3179.6555
Epoch 5/20
21105/21110 [============================>.] - ETA: 0s - loss: 9987745.0000 - root_mean_squared_error: 3160.3394

...

21110/21110 [==============================] - 54s 3ms/step - loss: 9555641.0000 - root_mean_squared_error: 3091.2200
Epoch 17/20
21103/21110 [============================>.] - ETA: 0s - loss: 9513987.0000 - root_mean_squared_error: 3084.4751

21110/21110 [==============================] - 54s 3ms/step - loss: 9514604.0000 - root_mean_squared_error: 3084.5752
Epoch 18/20
21092/21110 [============================>.] - ETA: 0s - loss: 9490068.0000 - root_mean_squared_error: 3080.5955

21110/21110 [==============================] - 54s 3ms/step - loss: 9490896.0000 - root_mean_squared_error: 3080.7297
Epoch 19/20
21095/21110 [============================>.] - ETA: 0s - loss: 9479045.0000 - root_mean_squared_error: 3078.8057

21110/21110 [==============================] - 55s 3ms/step - loss: 9479156.0000 - root_mean_squared_error: 3078.8237
Epoch 20/20
21094/21110 [============================>.] - ETA: 0s - loss: 9447482.0000 - root_mean_squared_error: 3073.6755

21110/21110 [==============================] - 55s 3ms/step - loss: 9448245.0000 - root_mean_squared_error: 3073.7998


In [86]:
for test in new_test:
    exist = False
    for train in new_train:
        if train == test:
            exist = True
    
    if not exist:
        new_test = new_test.drop([test], axis=1)

print(len(new_test.dtypes))
print(len(new_train.dtypes)-1) # beacause Sales is y data

19
19


In [88]:
import tensorflow as tf
test = new_test.to_numpy()
test = test.reshape((35104,1,19)) #fitting to size of input (3d array)
test.shape
test = tf.convert_to_tensor(test)

In [89]:
target = model1.predict(test)
prediction_data = pd.DataFrame(target)
prediction_data

1097/1097 [==============================] - 2s 1ms/step


,0
0,3951.515869
1,3964.346436
2,3977.176758
3,3990.007080
4,3907.065186
...,...
35099,3276.510498
35100,3282.158936
35101,3287.807373
35102,3293.455566


In [90]:
output = pd.DataFrame({'Id': test_data.Id, 'Sales': prediction_data[0]})
output.fillna(0, inplace=True)
output.to_csv('submission.csv', index=False)
output


,Id,Sales
0,1,3951.515869
1,2,3964.346436
2,3,3977.176758
3,4,3990.007080
4,5,3907.065186
...,...,...
41083,41084,0.000000
41084,41085,0.000000
41085,41086,0.000000
41086,41087,0.000000
